In [ ]:
import os
import gradio as gr
from langchain_mistralai import ChatMistralAI
from langchain_core.prompts import ChatPromptTemplate

# Инициализация модели Codestral
mistral_model = "codestral-latest"
llm = ChatMistralAI(model=mistral_model, temperature=0, api_key=os.environ["MISTRAL_API_KEY"])

In [ ]:
# Функция для взаимодействия с Codestral
def ask_codestral(user_input):
    prompt = ChatPromptTemplate.from_messages([
        ("user", user_input)
    ])
    chain = prompt | llm
    response = chain.invoke({})
    return response.content

In [6]:
#Vtune summary
import subprocess
import pandas as pd

# Путь к файлу .vtune
vtune_file_path = "E:/openmw/OMW_2/r002hs/r002hs.vtune"

output_csv = "summary.csv"
command = f"vtune -report summary -result-dir {vtune_file_path} -format csv -report-output {output_csv}"

subprocess.run(command, shell=True, check=True)


CompletedProcess(args='vtune -report summary -result-dir E:/openmw/OMW_2/r002hs/r002hs.vtune -format csv -report-output summary.csv', returncode=0)

In [7]:
# Чтение CSV-файла с результатами
df = pd.read_csv(output_csv)

In [12]:
# Путь к файлу .vtune
vtune_file_path = "E:/openmw/OMW_2/r002hs/r002hs.vtune"

# Название выходного CSV-файла
output_csv = "bottom_up_results.csv"

command = (
    f'vtune -report hotspots -r "{vtune_file_path}" -format=csv -report-output {output_csv}'
)

try:
    subprocess.run(command, shell=True, check=True)
    print(f"Данные успешно экспортированы в {output_csv}")
except subprocess.CalledProcessError as e:
    print(f"Ошибка при выполнении команды: {e}")

Данные успешно экспортированы в bottom_up_results.csv


In [ ]:
# Чтение CSV-файла
df = pd.read_csv(output_csv, sep='\t')

# Вывод первых строк таблицы
print("Bottom-Up результаты:")
print(df.head())

In [36]:
# Массив значений для фильтрации
filter_modules = ["openmw.exe", "osg.dll", "osgUtil.dll"]
# exclude_source_files = ["[Unknown]", "mutex", "thread", "xtree", "vector", "xhash", etc..]

filtered_df = df[
    df["Module"].isin(filter_modules) &
    (df["Source File"].str.endswith(".cpp") | df["Source File"].str.endswith(".hpp"))
]

output_csv_path = "filtered_results.csv"

filtered_df.to_csv(output_csv_path, sep="\t", index=False)

print(f"Отфильтрованные данные сохранены в {output_csv_path}")

Отфильтрованные данные сохранены в filtered_results.csv


In [37]:
import pandas as pd

# Путь к исходному файлу
input_csv_path = "filtered_results.csv"

# Чтение первых 5 строк
df = pd.read_csv(input_csv_path, sep="\t")
top_5_rows = df.head(5)

# Путь для нового файла
output_csv_path = "top_5_functions.csv"

# Запись в новый файл
top_5_rows.to_csv(output_csv_path, sep="\t", index=False)

In [3]:
import os

# Пути к директориям
apps_path = r"E:\openmw\apps\openmw"
components_path = r"E:\openmw\components"

# Функция для получения списка папок
def get_folders(directory):
    folders = []
    for item in os.listdir(directory):
        item_path = os.path.join(directory, item)
        if os.path.isdir(item_path):
            folders.append(item)
    return folders

# Получение списка папок в apps
apps_folders = get_folders(apps_path)
print(f"Папки в {apps_path}:")
for folder in apps_folders:
    print(f"  - {folder}")

# Получение списка папок в components
components_folders = get_folders(components_path)
print(f"\nПапки в {components_path}:")
for folder in components_folders:
    print(f"  - {folder}")

Папки в E:\openmw\apps\openmw:
  - mwbase
  - mwclass
  - mwdialogue
  - mwgui
  - mwinput
  - mwlua
  - mwmechanics
  - mwphysics
  - mwrender
  - mwscript
  - mwsound
  - mwstate
  - mwworld

Папки в E:\openmw\components:
  - bgsm
  - bsa
  - bullethelpers
  - compiler
  - config
  - contentselector
  - crashcatcher
  - debug
  - detournavigator
  - esm
  - esm3
  - esm4
  - esmloader
  - esmterrain
  - fallback
  - files
  - fontloader
  - fx
  - interpreter
  - l10n
  - loadinglistener
  - lua
  - lua_ui
  - misc
  - myguiplatform
  - navmeshtool
  - nif
  - nifbullet
  - nifosg
  - platform
  - process
  - resource
  - sceneutil
  - sdlutil
  - serialization
  - settings
  - shader
  - sqlite3
  - std140
  - stereo
  - terrain
  - testing
  - to_utf8
  - translation
  - version
  - vfs
  - widgets


In [1]:
import pandas as pd
import re
# Путь к CSV-файлу
input_csv_path = "top_5_functions.csv"

# Чтение CSV-файла
df = pd.read_csv(input_csv_path, sep="\t")

def create_function_structure(row):
    function_full = row["Function (Full)"]
    parts = function_full.split("::", maxsplit=2) 
    
    folder_name = parts[0].lower()
    class_name = parts[1]
    function_name = parts[2]

    # Убираем всё, что в скобках, включая сами скобки
    function_name = re.sub(r"\(.*?\)", "", function_name).strip()

    # Если function_name содержит шаблонные параметры, оставляем только имя функции
    if "<" in function_name:
        function_name = function_name.split("::")[-1]

    source_file = row["Source File"]
    module = row["Module"]
    
    # Создание структуры
    function_structure = {
        "folder_name": folder_name,
        "full_folder_name": "",
        "class_name" : class_name,
        "function_name": function_name,
        "source_file": source_file,
        "module": module,
        "return_type": "",  # Пока оставляем пустым
        "arguments": "",  # Пока оставляем пустым
        "code": ""  # Пока оставляем пустым
    }
    return function_structure

# Применение функции к каждой строке DataFrame
functions_list = []
for index, row in df.iterrows():
    function_structure = create_function_structure(row)
    functions_list.append(function_structure)

# Вывод результата
for function in functions_list:
    print(function)

{'folder_name': 'sceneutil', 'full_folder_name': '', 'class_name': 'PositionAttitudeTransform', 'function_name': 'accept', 'source_file': 'positionattitudetransform.hpp', 'module': 'openmw.exe', 'return_type': '', 'arguments': '', 'code': ''}
{'folder_name': 'mwrender', 'full_folder_name': '', 'class_name': 'IntersectionVisitorWithIgnoreList', 'function_name': 'skipTransform', 'source_file': 'renderingmanager.cpp', 'module': 'openmw.exe', 'return_type': '', 'arguments': '', 'code': ''}
{'folder_name': 'mwphysics', 'full_folder_name': '', 'class_name': 'PhysicsSystem', 'function_name': 'stepSimulation', 'source_file': 'physicssystem.cpp', 'module': 'openmw.exe', 'return_type': '', 'arguments': '', 'code': ''}
{'folder_name': 'sceneutil', 'full_folder_name': '', 'class_name': 'RigGeometry', 'function_name': 'cull', 'source_file': 'riggeometry.cpp', 'module': 'openmw.exe', 'return_type': '', 'arguments': '', 'code': ''}
{'folder_name': 'nifosg', 'full_folder_name': '', 'class_name': 'Valu

In [4]:
import os

# Функция для поиска совпадений и восстановления пути
def find_and_set_full_folder_name(function):
    folder_name = function["folder_name"]
    source_file = function["source_file"]

    # Поиск совпадений в apps_folders
    for folder in apps_folders:
        if folder_name.lower() == folder.lower():  # Сравниваем имена папок
            full_path = os.path.join(apps_path, folder, source_file)
            function["full_folder_name"] = full_path  # Добавляем полный путь в структуру
            print(f"Найдено совпадение в apps: {full_path}")
            return  # Выход из функции после первого совпадения

    # Если в apps_folders совпадение не найдено, ищем в components_folders
    for folder in components_folders:
        if folder_name.lower() == folder.lower():  # Сравниваем имена папок
            full_path = os.path.join(components_path, folder, source_file)
            function["full_folder_name"] = full_path  # Добавляем полный путь в структуру
            print(f"Найдено совпадение в components: {full_path}")
            return  # Выход из функции после первого совпадения

# Перебор functions_list и поиск совпадений
for function in functions_list:
    find_and_set_full_folder_name(function)

Найдено совпадение в components: E:\openmw\components\sceneutil\positionattitudetransform.hpp
Найдено совпадение в apps: E:\openmw\apps\openmw\mwrender\renderingmanager.cpp
Найдено совпадение в apps: E:\openmw\apps\openmw\mwphysics\physicssystem.cpp
Найдено совпадение в components: E:\openmw\components\sceneutil\riggeometry.cpp
Найдено совпадение в components: E:\openmw\components\nifosg\controller.hpp


In [5]:
import os
import re

# Функция для заполнения return_type, arguments и code
def extract_function_details(function):
    # Формирование полного пути к файлу
    file_path = function["full_folder_name"]

    # Чтение содержимого файла
    try:
        with open(file_path, "r", encoding="utf-8") as file:
            content = file.read()
    except FileNotFoundError:
        print(f"Файл не найден: {file_path}")
        return

    function_name = function["function_name"]

    # Регулярное выражение для поиска функции (включая возвращаемый тип и аргументы)
    function_pattern = re.compile(rf"(.*?)\b{function_name}\s*\(([^)]*)\)\s*{{")

    # Поиск функции
    function_match = function_pattern.search(content)
    if not function_match:
        print(f"Функция {function_name} не найдена в файле {file_path}.")
        return

    # Извлечение return_type и arguments из совпадения
    return_type = function_match.group(1).strip()  # Возвращаемый тип
    arguments = function_match.group(2).strip()   # Аргументы

    # Начальная позиция функции (после сигнатуры)
    start_pos = function_match.end()

    # Счётчик скобок
    brace_count = 1  # Начинаем с 1, так как первая скобка уже найдена
    function_code = function_match.group(0)  # Начинаем с сигнатуры функции
    i = start_pos

    # Итерация по символам для поиска конца функции
    while i < len(content):
        if content[i] == "{":
            brace_count += 1
        elif content[i] == "}":
            brace_count -= 1

        function_code += content[i]

        # Если все скобки закрыты, завершаем чтение
        if brace_count == 0:
            break

        i += 1

    # Заполнение полей return_type, arguments и code
    function["return_type"] = return_type
    function["arguments"] = arguments
    function["code"] = function_code

# Пример использования
for function in functions_list:
    extract_function_details(function)
    print(function)
    


Функция accept не найдена в файле E:\openmw\components\sceneutil\positionattitudetransform.hpp.
{'folder_name': 'sceneutil', 'full_folder_name': 'E:\\openmw\\components\\sceneutil\\positionattitudetransform.hpp', 'class_name': 'PositionAttitudeTransform', 'function_name': 'accept', 'source_file': 'positionattitudetransform.hpp', 'module': 'openmw.exe', 'return_type': '', 'arguments': '', 'code': ''}


KeyboardInterrupt: 

In [ ]:
# Формирование полного пути к файлу
file_path = function["full_folder_name"]

# Чтение содержимого файла
try:
    with open(file_path, "r", encoding="utf-8") as file:
        content = file.read()
except FileNotFoundError:
    print(f"Файл не найден: {file_path}")
    exit()

# Поиск класса и функции в исходном коде
class_name = function["class_name"]
function_name = function["function_name"]

# Регулярное выражение для поиска начала класса (только по имени)
class_start_pattern = re.compile(rf"class\s+{class_name}\b[^{{]*")

# Поиск начала класса
class_start_match = class_start_pattern.search(content)
if not class_start_match:
    print(f"Класс {class_name} не найден в файле {file_path}.")
    exit()

# Начальная позиция класса (после объявления)
start_pos = class_start_match.end()

# Счётчик скобок
brace_count = 0  # Начинаем с 0, так как скобка ещё не найдена
class_code = class_start_match.group(0)  # Начинаем с объявления класса
i = start_pos

# Итерация по символам для поиска конца класса
while i < len(content):
    if content[i] == "{":
        brace_count += 1
    elif content[i] == "}":
        brace_count -= 1
    class_code += content[i]
    # Если все скобки закрыты, завершаем чтение
    if brace_count == 0:
        break
    i += 1

# Вывод информации о классе и его кода
print(f"Информация о классе:\n{class_code}\n")

# Регулярное выражение для поиска функции (только по имени)
function_pattern = re.compile(rf"\b{function_name}\s*[^{{]*")

# Поиск функции
function_match = function_pattern.search(class_code)
if not function_match:
    print(f"Функция {function_name} не найдена в классе {class_name}.")
    exit()

# Начальная позиция функции (после сигнатуры)
start_pos = function_match.end()

# Счётчик скобок
brace_count = 0  # Начинаем с 0, так как скобка ещё не найдена
function_code = function_match.group(0)  # Начинаем с сигнатуры функции
i = start_pos

lenC = len(class_code)
# Итерация по символам для поиска конца функции
while i < len(class_code):
    if class_code[i] == "{":
        brace_count += 1
    elif class_code[i] == "}":
        brace_count -= 1

    function_code += class_code[i]

    # Если все скобки закрыты, завершаем чтение
    if brace_count == 0:
        break

    i += 1

# Вывод информации о функции и её кода
print(f"Код функции {function_name}:\n{function_code}\n")

In [6]:
import sqlite3

# Подключение к базе данных (или создание новой)
conn = sqlite3.connect("functions.db")
cursor = conn.cursor()

# Создание таблицы
cursor.execute("""
CREATE TABLE functions (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    folder_name TEXT NOT NULL,
    full_folder_name TEXT NOT NULL,
    class_name TEXT NOT NULL,
    function_name TEXT NOT NULL,
    source_file TEXT NOT NULL,
    module TEXT NOT NULL,
    return_type TEXT,
    arguments TEXT,
    code TEXT
)
""")
conn.commit()

In [7]:
# Вставка данных
for function in functions_list:  
    cursor.execute("""
    INSERT INTO functions (
        folder_name, full_folder_name, class_name, function_name, source_file, module, return_type, arguments, code
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, (
        function["folder_name"],
        function["full_folder_name"],
        function["class_name"],
        function["function_name"],
        function["source_file"],
        function["module"],
        function["return_type"],
        function["arguments"],
        function["code"]
    ))
conn.commit()

In [8]:
function_name = "skipTransform"
cursor.execute("SELECT * FROM functions WHERE function_name = ?", (function_name,))
result = cursor.fetchone()

if result:
    print(f"Найдена функция: {result}")
else:
    print("Функция не найдена.")

Найдена функция: (2, 'mwrender', 'E:\\openmw\\apps\\openmw\\mwrender\\renderingmanager.cpp', 'IntersectionVisitorWithIgnoreList', 'skipTransform', 'renderingmanager.cpp', 'openmw.exe', '', 'osg::Transform& transform', 'skipTransform(osg::Transform& transform)\n        {\n            if (mContainsPagedRefs)\n                return false;\n\n            osg::UserDataContainer* userDataContainer = transform.getUserDataContainer();\n            if (!userDataContainer)\n                return false;\n\n            for (unsigned int i = 0; i < userDataContainer->getNumUserObjects(); ++i)\n            {\n                if (PtrHolder* p = dynamic_cast<PtrHolder*>(userDataContainer->getUserObject(i)))\n                {\n                    if (std::find(mIgnoreList.begin(), mIgnoreList.end(), p->mPtr) != mIgnoreList.end())\n                    {\n                        return true;\n                    }\n                }\n            }\n\n            return false;\n        }\n\n        voi

In [ ]:
# Создаем интерфейс Gradio
interface = gr.Interface(
    fn=ask_codestral,
    inputs=gr.Textbox(lines=2, placeholder="Введите ваш запрос..."),
    outputs=gr.Textbox(lines=10, label="Ответ Codestral"),
    title="Codestral Code Assistant",
    description="Задайте вопрос или попросите написать код, и Codestral поможет вам!"
)

In [ ]:
# Запускаем интерфейс
interface.launch()